In [1]:
from IPython.display import Image

# 글자 단위 언어 모델 구현
* 언어 모델링: 영어 문장 생성처럼 기계가 사람의 언어와 관련된 작업을 수행하도록 만드는 애플리케이션.
* 모델의 입력 = 텍스트 문서
    * 입력 문서와 비슷한 스타일로 새로운 텍스트를 생성하는 모델을 만드는 것
    * ex) 책, 특정 프로그래밍 언어 등
    * 글자의 시퀀스로 나뉘어 한 번에 한 글자씩 네트워크에 주입
        * 네트워크는 지금까지 본 글자와 함께 새로운 글자를 처리하여 다음 글자를 예측


In [2]:
# 글자 단위 언어 모델링
# EOS: 시퀀스의 끝을 의미(end of sequence)
Image(url='https://git.io/JLdVE', width=700)

#### 데이터셋 전처리
* 글자 수준의 언어 모델링을 위한 데이터 준비
    * 구텐베르크 프로젝트 웹 사이트에서 입력 데이터를 구한다.
    * 쥘 베튼이 1874년에 출간한 <신비의 섬> 책의 텍스트를 사용
        * http://www.gutenberg.org/files/1268/1268-0.txt
        * curl -O http://www.gutenberg.org/files/1268/1268-0.txt

In [3]:
!wget https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt

--2022-02-17 08:35:55--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171600 (1.1M) [text/plain]
Saving to: ‘1268-0.txt’

1268-0.txt          100%[===================>]   1.12M  --.-KB/s    in 0.05s   

2022-02-17 08:35:56 (22.4 MB/s) - ‘1268-0.txt’ saved [1171600/1171600]



In [4]:
import numpy as np

# 텍스트를 읽어 전처리
with open("1268-0.txt", 'r', encoding='UTF8') as fp:
    text = fp.read()

print(text[:50])

﻿The Project Gutenberg EBook of The Mysterious Isl


In [5]:
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이: ', len(text))
print('고유 문자 길이: ', len(char_set))

567 1112917
전체 길이:  1112350
고유 문자 길이:  80


대부분의 신경망 라이브러리와 RNN 구현은 문자열 형태의 입력 데이터를 다룰 수 없다. 그래서 텍스트 데이터를 숫자 형태로 매핑해야하기 때문에 파이썬 딕셔너리 char2int를 만들어 각 문자를 정수로 매핑한다. 또한, 모델의 출력 결과를 텍스트로 변환하는 역 매핑도 필요하다. 정수와 문자를 키와 값으로 연결한 딕셔너리로 역 매핑을 수행할 수 있지만 인덱스와 고유 문자를 매핑한 넘파이 배열을 사용하는 것이 훨씬 효율적이다.

In [6]:
# 문자와 인덱스 매핑
Image(url='https://git.io/JLdVz', width=700)

In [7]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('인코딩된 텍스트 크기: ', text_encoded.shape)

print(text[:15], '      == 인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], '  == 디코딩 ==>', ''.join(char_array[text_encoded[15:21]]))

인코딩된 텍스트 크기:  (1112350,)
THE MYSTERIOUS        == 인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]   == 디코딩 ==> ISLAND


넘파이 배열 text_encoded는 텍스트에 있는 모든 문자에 대한 인코딩 값을 담고 있다. 이 배열을 사용하여 텐서플로 데이터셋을 만든다.

In [8]:
import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


텍스트에 나타난 순서대로 문자를 담기 위해 반복 가능한 Dataset 객체를 만들었다.

In [9]:
# 불완전한 문자 시퀀스 집합에 텍스트 생성의 작업으로 Targets을 예측
Image(url='https://git.io/JLdVV', width=700)

불완전한 문자 시퀀스에 대해 다음 문자를 예측해야 한다. 80개의 고유 문자가 있으므로 이 문제는 다중 분류 작업이 된다.

In [10]:
# 다중 분류 방식을 기반으로 길이 1인 시퀀스(한 글자)로 시작해서 새로운 텍스트를 반복하여 생성
# 새로운 텍스트 생성
Image(url='https://git.io/JLdVr', width=700)

텐서플로로 텍스트 생성 모델을 구현하기 위해 시퀀스 길이를 40으로 잘라 입력 텐서 x가 40개의 토큰으로 구성되도록 한다. 실제로 시퀀스 길이는 생성된 텍스트 품질에 영향을 미친다. 긴 시퀀스가 더 의미 있는 문장르 만들 수 있다. 하지만 짧은 시퀀스일 경우 모델이 대부분의 문맥을 무시하고 개별 단어를 정확히 감지하는 데 초점을 맞출 수 있다.

긴 시퀀스가 보통 더 의미 있는 문장을 만들지만 긴 시퀀스에서 RNN모델이 장기간 의존성을 감지하기 어렵다. 따라서 실제로 **적절한 시퀀스 길이를 찾는 것은 경험적으로 평가해야 하는 하이퍼파라미터 최적화 문제**이다.

위 그림에서 보듯 입력 x와 타깃 y는 한 글자씩 어긋나 있다. 따라서 텍스트는 41 문자씩나눈다. 처음부터 40개의 문자는 입력 시퀀스 x가 되고 마지막 문자는 타깃 시퀀스 y가 된다.

Dataset 객체 ds_text_encoded에 인코딩된 전체 텍스르르 원본 문자 순서대로 저장해놓았다. batch() 메서드를 사용하여 41개의 문자로 구성된 텍스트 조각을 만들고, 마지막 배치에서 길이가 41보다 작으면 이 배치는 버린다. 즉, 데이터셋은 항상 길이가 41인 시퀀스를 담고 있다. 이 41개의 문자 조각을 사용해서 시퀀스 x(입력)와 시퀀스 y(타깃)을 만든다. 두 시퀀스는 모두 40개의 원소로 구성된다.

In [11]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

# inspection
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


In [12]:
# x & y를 나누기 위한 함수
def split_input_target(chunk):
    input_seq = chunk[:-1] # 0 ~ 40
    target_seq = chunk[1:] # 1 ~ 41
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

# 확인
for ex in ds_sequences.take(2):
    print('입력 (x):', repr(''.join(char_array[ex[0].numpy()])))
    print('입력 (x):', repr(''.join(char_array[ex[1].numpy()])))
    print()

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
입력 (x): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
입력 (x): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



데이터셋 준비의 마지막 단계로 이 데이터셋을 미니 배치로 나눈다. 데이터셋을 배치로 나누기 위해 첫 번째 전처리 단계에서 문장 조각을 만들었다. 각 조각이 하나의 훈련 샘플을 대응하는 문장을 표현한다.

In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE) # drop_remainder=True)
ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))>

#### 문자 수준의 RNN 모델

In [14]:
# 코드 재사용을 위해 케라스 Sequential 클래스로 모델을 만드는 함수
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units=512

tf.random.set_seed(1)

model = build_model(
    charset_size,
    embedding_dim,
    rnn_units)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         20480     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


* LSTM 층의 출력 크기 = (None, None, 512), 랭크 3
    * 첫 번째 차원: 배치 차원
    * 두 번째 차원: 출력 시퀀스 길이
    * 마지막 차원: 은닉 유닛의 개수
    * 랭크 3의 출력을 만드는 이유
        * LSTM 층을 만들 때 return_sequences=True로 지정했기 때문이다.
        * 완전 연결 층(Dense)이 LSTM 층의 출력을 받아 출력 시퀀스의 각 원소마다 로짓을 계산한다. 결국 이 모델의 최종 출력도 랭크 3 텐서가 된다.
    
* 마지막 완전 연결 층을 activation=None 으로 설정
    * 새로운 텍스트를 생성하기 위해 모델 예측 값에서 샘플링할 수 있도록 로짓 출력이 필요하기 때문이다.

In [16]:
# 모델 훈련
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))
model.fit(ds, epochs=20)

Epoch 1/20
424/424 [==============================] - 28s 36ms/step - loss: 2.3212
Epoch 2/20
424/424 [==============================] - 16s 36ms/step - loss: 1.7581
Epoch 3/20
424/424 [==============================] - 16s 35ms/step - loss: 1.5524
Epoch 4/20
424/424 [==============================] - 16s 35ms/step - loss: 1.4330
Epoch 5/20
424/424 [==============================] - 16s 37ms/step - loss: 1.3577
Epoch 6/20
424/424 [==============================] - 16s 35ms/step - loss: 1.3056
Epoch 7/20
424/424 [==============================] - 16s 35ms/step - loss: 1.2656
Epoch 8/20
424/424 [==============================] - 16s 35ms/step - loss: 1.2340
Epoch 9/20
424/424 [==============================] - 16s 36ms/step - loss: 1.2086
Epoch 10/20
424/424 [==============================] - 16s 35ms/step - loss: 1.1860
Epoch 11/20
424/424 [==============================] - 16s 35ms/step - loss: 1.1665
Epoch 12/20
424/424 [==============================] - 19s 44ms/step - loss: 1.1487
E

#### 평가: 새로운 텍스트 생성

훈련한 RNN 모델은 각 문자에 대해 80개 크기의 로짓을 반환한다. **소프트맥스 함수를 사용해 이 로짓을 쉽게 확률**로 바꿀 수 있다. 이 확률을 사용하여 다음 문자를 결정한다. 시퀀스에서 다음 문자를 예측하기 위해 간단히 가장 큰 로짓 값을 가진 원소를 선택할 수 있다. 하지만 항상 가장 높은 확률을 가진 문자를 선택하는 대신 **출력에서 (랜덤하게) 샘플링**하려고 한다. 이렇게 하지 않으면 모델이 항상 동일한 텍스트를 만든다.

텐서플로의 tf.random.categorical() 함수를 사용하여 범주형 분포에서 랜덤하게 샘플링할 수 있다.



In [17]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('확률: ', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률:  [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]])


로짓이 같아 이 범주는 동일한 확률을 가진다.(즉, 범위 선택 가능성이 동일) 따라서 샘플 크기가 크면 각 범주가 등장할 횟수는 샘플 크기의 1/3에 이를 것으로 기대할 수 있다. 로짓을 [1, 1, 3]으로 바꾸면 범주 2가 더 많이 등장할 것이다.

In [18]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])


tf.random.categorical 함수를 사용하면 모델이 추력한 로짓을 기반으로 문자를 생성할 수 있다. 짧은 시작 문자열 starting_str을 받아 새로운 generated_str을 생성하는 sample() 함수를 정의한다..



In [19]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1)) # 정수 시퀀스

    generated_str = starting_str # 초기의 입력 값으로 설정

    model.reset_states()
    for i in range(len_generated_text):
        # RNN 모델에 전달하여 로짓 계산, RNN 모델의 마지막 순환 층에서 
        # return_sequences=True로 설정해 입력 시퀀스와 동일한 길이의 로짓 시퀀스가 출력된다.
        # 따라서 RNN 모델의 출력에 있는 각 원소는 모델이 입력 시퀀스를 관찰해 다음 문자를 위한 로짓을 표현한다.(80개의 원소 벡터)
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        # 출력 logits의 마지막 원소만 tf.random.categorical() 함수로 전달하여 새로운 샘플 생성
        new_char_indx = tf.random.categorical(scaled_logits, num_samples=1)

        # 새로운 샘플을 문자로 변환하고 생성된 문자열 generated_text 끝에 추가하여 길이를 1만큼 늘린다.
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy() 
        generated_str += str(char_array[new_char_indx])

        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        # 지정한 문자 길이만큼 생성될 때까지 generated_text에서 마지막 max_input_length 개의 문자를 선택하고
        # 이를 사용해서 새로운 문자를 생성한다.
        encoded_input = encoded_input[:, -max_input_length:]
    
    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island was eggs in his most ary where a portion of this vessel nor
me, true to admire
ascending beyond of ears on each side operation, which contained the passengers which devoured them to be only forward and to speed again
by Pencroft.

“It is delia. In the perimeter of the past, was already passing from a new till threatelessul, but
the colonists fell back, which
was always received in any uneasy and calmer; but we
will been recalleed, this evening there was not without green meal, friend was for th


생성된 샘플의 예측 가능성을 조절하기 위해(즉, 생성된 텍스트가 훈련 텍스트에서 학습된 패턴을 따르게 할지 랜덤하게 생성할지 조절하기 위해) RNN이 계산한 로짓을 tf.random.categorical() 샘플링 함수로 전달하기 전에 스케일을 조정할 수 있다.

스케일링 인자 a를 물리학의 온도의 역수로 해석할 수 있다. 온도가 높으면 무작위성이 커지고 온도가 낮으면 예측 가능한 행동을 만든다.

a < 1로 로짓의 스케일을 조정하면 소프트맥스 함수가 계산할 확률은 더 균일해진다.

In [20]:
logits = np.array([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률: ', tf.math.softmax(logits).numpy()[0])

print('0.5배 조정 후 확률:  ', tf.math.softmax(0.5*logits).numpy()[0])

print('0.1배 조정 후 확률:  ', tf.math.softmax(0.1*logits).numpy()[0])

스케일 조정 전의 확률:  [0.10650698 0.10650698 0.78698604]
0.5배 조정 후 확률:   [0.21194156 0.21194156 0.57611688]
0.1배 조정 후 확률:   [0.31042377 0.31042377 0.37915245]


a = 0.1로 로짓의 스케일을 조정하면 거의 균등한 확률을 얻는다. 

In [21]:
# a=2.0 -> 예측 가능성이 높아짐
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island was extremely still trees, and a few months this very stream which they had once extremely profitical. The engineer was heard, and the colonists were allowed them to be surprised to the corral, and a few minutes he expected to the ocean, and as the reporter and Neb and Pencroft also accompanied the engineer, who was lost to be best to despair, since the lad was about to reply to the east. They had stailed on the
surface of the earth in the corral.

The colonists were obliged to reply to the int


In [22]:
# a=0.5 -> 무작위성이 높아짐
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island had egbloes hollig’t grye?

“He musmel!” cratcif-Cave band,
my, froj wembles, “and thart?” he remaces. No, enohes aboup
hig a;
unforciDent, yourse head, was
Towen under-siepplifin oyesior,’!

“We can, at lew var.
He

Who is how pehocious delbay rock to estage on.
Hadder
Neb! capabarly, wized he
would excet? Afre! we little inscotch
Ayr?” ?Hisfermening, whno Aferailiza,” her masside yol dwombled! Unlivilly 3uspur
her year.
They extremity very day--were-fall, with the rooms, dlen’s sanch.

AA9usi


a=0.5로 로짓의 스케일을 조정하면 더 랜덤한 텍스트가 생성된다. 올바른 텍스트와 신선한 텍스트 생성 사이에서 절충점을 찾아야 한다.

시퀀스-투-시퀀스(sequence-to-sequence, seq2seq) 모델링 작업인 문자 수준의 텍스트 생성 문제를 다루었다. 이것 자체로는 아주 유용하지는 않지만 이런 종류의 모델에 맞는 애플리케이션이 있다. 예를 들어 비슷한 RNN 모델을 훈련하여 간단한 질문에 답변하는 챗봇을 사용할 수 있다.

# 16.4 트랜스포머 모델을 사용한 언어 이해

* 트랜스포머: 입력과 출력 시퀀스 사이에 있는 전역 의존성(global dependency)을 모델링할 수 있다.
    * 트랜스포머 구조는 어텐션(attention)이라는 개념을 기반으로 한다.(셀프 어텐션 메커니즘)

### 16.4.1 셀프 어텐션 메커니즘 이해

#### 셀프 어텐션 기본 구조

* 입력 시퀀스: 길이가 T인 $x^{(0)}, x^{(1)}, ... , x^{(T)}$
* 출력 시퀀스: 길이가 T인 $O^{(0)}, O^{(1)}, ... , O^{(T)}$

시퀀스 각 원소 $x^{(t)}와 O^{(t)}$는 크기가 d인 벡터이다. 그 다음 seq2seq 작업에서 셀프 어텐션은 입력 원소에 대한 출력 시퀀스에 있는 각 원소의 의존성을 모델링하는 것이 목적이다. 이를 위해 어텐션 메커니즘은 세 단계로 구성된다.

1. 현재 원소와 시퀀스에 있는 다른 모든 원소 사이의 유사도를 기반으로 중요도 가중치를 계산
2. 소프트맥스 함수를 사용하여 이 가중치를 정규화
3. 이 가중치를 해당하는 시퀀스 원소와 결합하여 어텐션 값을 계산

식으로 나타내면 셀프 어텐션의 출력은 모든 입력 시퀀스의 가중치 합이다. 예를 들어 i번째 입력 원소에 해당하는 출력은

$$o^{(i)} = \sum_{j=0}^{T}W_{ij}x^{j}$$

여기서 가중치 $W_{ij}$는 현재 입력 원소 $x{i}$와 입력 시퀀스에 있는 다른 모든 원소 사이의 유사도를 기반으로 계산된다. 즉 이 유사도는 현재 입력 원소 $x^{(i)}$와 입력 시퀀스에 있는 다른 원소 $x^{(j)}$의 접곱으로 계산된다.

$$w_{ij}=x^{(i)^T}x^{(j)}$$

i번째 입력과 시퀀스에 있는 모든 입력($x^{(i)}$에서 $x^{(T)}$까지)에 대해 유사도 기반 가중치를 계산한 후 원본 가중치($w_{i0}$에서 $w_{iT}$까지)를 다음과 같이 소프트맥스 함수로 정규화한다.

$$W_{ij} = \frac{exp(w_{ij})}{\sum_{j=0}^Texp(w_{ij})} = softmax([w_{ij}]_{j=0..T})$$

소프트맥스 함수를 적용하기 때문에 정규화된 가중치 합은 1이 된다.

$$\sum^T_{j=0}W_{ij}=1$$

정리하면 셀프 어텐션 연산의 주요 세 단계는 다음과 같다.

1. 주어진 입력 원소 x^{(i)}와 [0, T] 범위에 있는 j번째 원소에 대해 점곱 $x^{(i)^T}x^{(j)}$를 계산
2. 소프트맥스 함수로 이 점곱을 정규화하여 가중치 $W_{ij}$를 얻는다
3. 전체 입력 시퀀스에 대한 가중치 합으로 출력 $O^{(i)}$를 계산



In [2]:
# 셀프 어텐션 과정
Image(url='https://git.io/JLdVo', width=700)

#### 쿼리, 키, 값 가중치를 가진 셀프 어텐션 메카니즘

언어 모델을 훈련할 때 분류 오차를 최소화하는 것 같이 목적 함수를 최적화하려면 입력 원소 $x^{(i)}$가 되는 단어 임베딩(즉, 입력 벡터)을 바꾸어야 한다. 다시 말해 기본적인 셀프 어텐션 메커니즘을 사용하면 트랜스포머 모델이 주어진 시퀀스에서 모델을 최적화하는 동안 어텐을 값을 바꾸거나 업데이트하는 데 제한적이다.

셀프 어텐션 메커니즘을 모델 최적화에 대해 유연하고 적응할 수 있게 만들기 위해 추가적인 가중치 행렬을 사용한다. 이 가중치는 모델을 훈련하는 동안 학습되는 모델 파라미터이다. 이 세 가지 가중치를 $U_q, U_k, U_v$로 표시한다. 이 가중치는 입력을 쿼리, 키, 값 시퀀스로 만들기 위해 사용된다.

* 쿼리 시퀀스: $q^{(i)}=U_qx^{(i)}$, $i \in [0, T]$일 때
* 키 시퀀스: $k^{(i)}=U_kx^{(i)}$, $i \in [0, T]$일 때
* 값 시퀀스: $v^{(i)}=U_vx^{(i)}$, $i \in [0, T]$일 때

여기서 $q^{(i)}$와 $k^{(i)}$는 크기가 $d_k$인 벡터이다. 따라서 투영된 행렬 $U_q$와 $U_k$의 크기는 $d_k * d$이다. 반면 $U_v$의 크기는 $d_v * d$이다. 예를 들어 이 벡터의 크기를 $m=d_k=d_v$로 동일할 때, 입력 시퀀스 원소 $x^{(i)}$와 j번째 시퀀스 원소 $x^{(j)}$ 사이에 접곱으로 정규화되지 않은 가중치를 계산하는 대신에 쿼리와 키 사이에 점곱을 계산한다.

$$w_{ij}=q^{(i)^T}k^{(j)}$$

그다음 소프트맥스 함수로 가중치 $w_{ij}$를 정규화하기 전에 $1/\sqrt{m}$로 스케일을 조정한다.

$$W_{ij}=softmax(\frac{w_{ij}}{\sqrt{m}})$$

$1/\sqrt{m}$로 $w_{ij}$의 스케일을 조정하면 가중치 벡터의 유클리드 길이가 거의 같은 범위에 있게 된다.

### 16.4.2 멀티-헤드 어텐션과 트랜스포머 블록

셀프 어텐션 메커니즘의 판별 능력을 크게 높이는 또 다른 기술은 여러 개의 셀프 어텐션 연산을 합친 **멀티 헤드 어텐션**(Multi-Head Attention, MHA)이다. 이 경우 각 셀프 어텐션 메커니즘을 헤드(head)라고 부르며 병렬로 계산할 수 있다. r개의 병렬 헤드를 사용하여 각 헤드는 크기가 m인 벡터 h를 만든다. 이 벡터를 연결하여 크기가 r * m인 벡터 z를 얻는다. 마지막으로 이 연결된 벡터와 출력 행렬 $W^o$를 접곱하여 다음과 같이 최종 출력을 만든다.

$$o^{(i)}=W_{ij}^oz$$

In [3]:
# 트랜스포머 블록 구조
Image(url='https://git.io/JLdV6', width=700)

위 그림에 나온 트랜스포머 구조에는 언급하지 않은 두 가지 구성 요소가 추가되어 있다.

이 중 하나는 **잔차 해결**(residual connection)이다. 잔차 연결은 층(또는 층 그룹)의 출력을 입력에 더한다. 즉, x + layer(x)이다. 잔차 연결로 층(또는 층 그룹)을 구성하는 블록을 **잔차 블록**(residual block)이라고 한다. 위 그림의 트랜스포머 블록은 두 개의 잔차 블록을 포함한다.

다른 하나는 **층 정규화**(layer normalization)이다. 정규화 층의 한 종류이다. 

트랜스포머 모델은 먼저 입력 시퀀스가 셀프 어텐션 메커니즘을 기반으로 하는 MHA 층으로 전달된다. 또한, 입력 시퀀스가 잔차 연결을 통해 MHA 층의 출력에 더해진다. 이렇게 하면 훈련하는 동안 앞쪽의 층이 충분한 그레이디언트 신호를 받게 된다. 훈련 속도와 수렴을 향상시키기 위해 자주 사용하는 기법이다.

입력 시퀀스가 MHA 층의 출력에 더해진 후 이 출력이 층 정규화를 통해 정규화된다. 정규화된 신호가 연속된 MLP(완전 연결) 층과 잔차 연결을 통과한다. 마지막으로 잔차 블록의 출력을 다시 정규화하여 출력 시퀀스로 반환해서 시퀀스 분류나 시퀀스 생성에 사용할 수 있다.